## Translate Montana (MT) stream gage data into Geoconnex


Sources are: https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/0 for the locations, for which copy features is performed and https://gis.dnrc.mt.gov/apps/stage/gage-report/ and for data 

In [44]:
import arcpy

# Set the workspace environment
arcpy.env.workspace = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\GeoconnexNPDES.gdb"

# reassign variable name
MT_gage = "MT_gages"

# Define a list of field names and their types
field_list = [
    ("uri", "TEXT"),
    ("name", "TEXT"),
    ("id", "TEXT"),
    ("provider_name", "TEXT"),
    ("provider_id", "TEXT"),
    ("provider_url", "TEXT"),
    ("provider_code", "TEXT"),
    ("mainstem_uri", "TEXT"),
    ("comid", "TEXT")
]

# Add the new fields using a loop
for field_name, field_type in field_list:
    arcpy.AddField_management(MT_gage, field_name, field_type)

# Confirm the field addition
field_names = [field.name for field in arcpy.ListFields(MT_gage)]
print("Fields in the layer after addition:", field_names)

Fields in the layer after addition: ['OBJECTID', 'Shape', 'LocationID', 'LocationCode', 'LocationName', 'LocationPath', 'LocationType', 'IsExternalLocation', 'Longitude', 'Latitude', 'UtcOffset', 'LastModified', 'ElevationUnits', 'Elevation', 'Description', 'Tags', 'ExtendedAttributeValues', 'StatusDesc', 'ActiveFlag', 'CountyName', 'BasinName', 'HUC8Code', 'uri', 'name', 'id', 'provider_name', 'provider_id', 'provider_url', 'provider_code', 'mainstem_uri', 'comid']


In [45]:
## Get unique ID, which comes from Site_Name

# Create an update cursor to iterate through the rows
with arcpy.da.UpdateCursor(MT_gage, ['LocationCode', 'id']) as cursor:
    for row in cursor:
        # Get the Site_Name value
        site_name = row[0]

        # Remove spaces and assign the result to the 'id' field
        row[1] = site_name.replace(" ", "")

        # Update the row
        cursor.updateRow(row)

# Clean up the cursor
del cursor

# Set name = id
# Define the expression for the field calculation
expression = "!id!"  # Set the 'name' field equal to the 'id' field

# Perform the field calculation
arcpy.management.CalculateField(MT_gage, "name", expression, "PYTHON3")

print("id and name fields populated")

id and name fields populated


In [46]:
# Confirm unique ID in identifier field
# Field to check for uniqueness
field_name = "id"

# Use a set to keep track of unique values encountered in the field
unique_values = set()

# Initialize a counter to keep track of the total number of rows
total_rows = 0

# Initialize a flag to check for spaces
has_spaces = False

# Start a search cursor to iterate through the rows and check for unique values and spaces
with arcpy.da.UpdateCursor(MT_gage, field_name) as cursor:
    for row in cursor:
        total_rows += 1
        value = row[0]
        if value in unique_values:
            print(f"Non-unique value found: {value}")
        else:
            unique_values.add(value)

        if " " in value:
            has_spaces = True
            # Replace spaces with "-"
            row[0] = value.replace(" ", "-")
            cursor.updateRow(row)

# Check if the number of unique values is the same as the total number of rows
if len(unique_values) == total_rows:
    print("All values in the 'Identifier' field are unique.")
else:
    print("There are duplicate values in the 'Identifier' field.")

# Check if there are any spaces in the 'Identifier' field
if has_spaces:
    print("There were spaces in the 'Identifier' field. They have been replaced with '-'.")
else:
    print("No spaces found in the 'Identifier' field.")


All values in the 'Identifier' field are unique.
No spaces found in the 'Identifier' field.


In [47]:
#Update Fields

arcpy.CalculateField_management(MT_gage, "provider_name", "'Montana Department of Natural Resources and Conservation'")
#Updated to Montana
arcpy.CalculateField_management(MT_gage, "provider_url", '"https://gis.dnrc.mt.gov/apps/stage/"', "PYTHON3")
arcpy.CalculateField_management(MT_gage, "provider_id", '"{}"'.format('"!id!"'), "PYTHON3")
arcpy.CalculateField_management(MT_gage, "provider_code", "'mtdnrc'")

print("Fields calculated")

Fields calculated


In [48]:
#create url
codeblock = """
def url_join(*parts: list) -> str:
    return '/'.join([str(p).strip().strip('/') for p in parts])
"""
##geoconnex.us/ndwr/gages/id

expression = """url_join("https://geoconnex.us/mtdnrc/gages", !id!)"""
arcpy.management.CalculateField(MT_gage, "uri", expression, "PYTHON3", codeblock)
print("uri calculated")



uri calculated


### Read in data table from https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3 service for secondary data table


In [49]:
import requests
import pandas as pd
# Define the URL of the REST API endpoint
url = "https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query"
# Define the parameters for your request
params = {
    "where": "1=1",  # This is a simple query to get all records
    "outFields": "*",  # This will include all fields in the response
    "f": "json",  # You can request data in JSON format
}
# Make a GET request to the API
response = requests.get(url, params=params)
# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Extract the features from the response
    features = [v['attributes'] for v in data['features']]

    df = pd.json_normalize(features)
        
    # Specify the full path for saving the CSV file
    save_path = r'C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_stations_data.csv'
      
    # Save the DataFrame to the specified CSV file
    df.to_csv(save_path, index=False)
    print(f"Data downloaded and saved as '{save_path}'")
else:
    print("Failed to retrieve data. Status code:", response.status_code)

Data downloaded and saved as 'C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_stations_data.csv'


In [50]:
df.head()

,OBJECTID,SensorCode,SensorID,SensorLabel,Comment,Description,isPublished,LocationName,LocationCode,LocationID,SubLocationCode,TimeSeriesType,Parameter,UnitOfMeasure,UtcOffset,ComputationMethod,ComputationPeriod,LastModifiedTime,ExtendedAttributeValues,ParameterLabel
0,473,Discharge.Daily Average@40A 4080,2a35ab6f412e42eca0ee527848845c04,Daily Average,,,1,Martinsdale Supply Canal,40A 4080,35143516212943bf8c60b01287ad82f0,None,ProcessorDerived,QR,ft^3/s,-PT7H,Mean,Daily,1612247042000,[],Discharge
1,489,Discharge.discharge@40A 9000,7c7b10a3025141db97c7038dfcf3f72d,discharge,,,1,Careless Canal,40A 9000,1ada4ab23774452797a7060a7fe11070,None,ProcessorDerived,QR,ft^3/s,-PT7H,Unknown,Unknown,1612247043000,[],Discharge
2,505,Stage.working@41K 4000,5ba899971ba6499ebc5e01ff59756005,working,,,1,Nilan East Canal blw Reservoir,41K 4000,b6b206ab5b5f423eb38697d591588727,None,ProcessorDerived,HG,ft,-PT7H,Unknown,Unknown,1612315443000,[],Stage
3,521,Stage.working-old@41H 2000,96bb4ce89b574d918aadd66c74e3c867,working-old,,,1,Middle Creek blw Res,41H 2000,5f1794586cda4561a3b0dc048e858ed3,None,ProcessorDerived,HG,ft,-PT7H,Unknown,Unknown,1612247045000,[],Stage
4,537,Discharge.Daily Average@76E 3000,7167bd3b11b44a09b7c91be2718abed8,Daily Average,,,1,EF Rock Main Canal abv Siphon,76E 3000,53c7781e54cb40ce87683ecf74e17a78,None,ProcessorDerived,QR,ft^3/s,-PT7H,Mean,Daily,1612247046000,[],Discharge


In [51]:
# Assuming you have already loaded data into the DataFrame 'df'

#get unique ids for the site name from location code, but remove spaces
df['Site_name'] = df['LocationCode']
# Remove spaces in the "Site_name" field
df['Site_name'] = df['Site_name'].str.replace(' ', '')
# this is where I need to check 
df["about_uri"] = "https://geoconnex.us/mtdnrc/gages/" + df['Site_name']

# # Update format of Montana gages using a lambda function
df["url"] = df["SensorCode"].apply(lambda x: f"https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='{x.replace(' ', '+')}'")

#Calculate other fields 
df["provider_code"] = "mtdnrc"
df["name"] = df["SensorCode"]
df["parameter_id"] = df["provider_code"] + "-" + df["ParameterLabel"]
df["parameter_name"] = df['SensorCode'].str.split('@').str[0]
df["parameter_group"] = df["ParameterLabel"]

print("Complete")


Complete


In [52]:
df.head()

,OBJECTID,SensorCode,SensorID,SensorLabel,Comment,Description,isPublished,LocationName,LocationCode,LocationID,SubLocationCode,TimeSeriesType,Parameter,UnitOfMeasure,UtcOffset,ComputationMethod,ComputationPeriod,LastModifiedTime,ExtendedAttributeValues,ParameterLabel,Site_name,about_uri,url,provider_code,name,parameter_id,parameter_name,parameter_group
0,473,Discharge.Daily Average@40A 4080,2a35ab6f412e42eca0ee527848845c04,Daily Average,,,1,Martinsdale Supply Canal,40A 4080,35143516212943bf8c60b01287ad82f0,None,ProcessorDerived,QR,ft^3/s,-PT7H,Mean,Daily,1612247042000,[],Discharge,40A4080,https://geoconnex.us/mtdnrc/gages/40A4080,https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.Daily+Average@40A+4080',mtdnrc,Discharge.Daily Average@40A 4080,mtdnrc-Discharge,Discharge.Daily Average,Discharge
1,489,Discharge.discharge@40A 9000,7c7b10a3025141db97c7038dfcf3f72d,discharge,,,1,Careless Canal,40A 9000,1ada4ab23774452797a7060a7fe11070,None,ProcessorDerived,QR,ft^3/s,-PT7H,Unknown,Unknown,1612247043000,[],Discharge,40A9000,https://geoconnex.us/mtdnrc/gages/40A9000,https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.discharge@40A+9000',mtdnrc,Discharge.discharge@40A 9000,mtdnrc-Discharge,Discharge.discharge,Discharge
2,505,Stage.working@41K 4000,5ba899971ba6499ebc5e01ff59756005,working,,,1,Nilan East Canal blw Reservoir,41K 4000,b6b206ab5b5f423eb38697d591588727,None,ProcessorDerived,HG,ft,-PT7H,Unknown,Unknown,1612315443000,[],Stage,41K4000,https://geoconnex.us/mtdnrc/gages/41K4000,https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Stage.working@41K+4000',mtdnrc,Stage.working@41K 4000,mtdnrc-Stage,Stage.working,Stage
3,521,Stage.working-old@41H 2000,96bb4ce89b574d918aadd66c74e3c867,working-old,,,1,Middle Creek blw Res,41H 2000,5f1794586cda4561a3b0dc048e858ed3,None,ProcessorDerived,HG,ft,-PT7H,Unknown,Unknown,1612247045000,[],Stage,41H2000,https://geoconnex.us/mtdnrc/gages/41H2000,https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Stage.working-old@41H+2000',mtdnrc,Stage.working-old@41H 2000,mtdnrc-Stage,Stage.working-old,Stage
4,537,Discharge.Daily Average@76E 3000,7167bd3b11b44a09b7c91be2718abed8,Daily Average,,,1,EF Rock Main Canal abv Siphon,76E 3000,53c7781e54cb40ce87683ecf74e17a78,None,ProcessorDerived,QR,ft^3/s,-PT7H,Mean,Daily,1612247046000,[],Discharge,76E3000,https://geoconnex.us/mtdnrc/gages/76E3000,https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.Daily+Average@76E+3000',mtdnrc,Discharge.Daily Average@76E 3000,mtdnrc-Discharge,Discharge.Daily Average,Discharge


In [53]:
# Print the first five rows of the "data_set_url" column
pd.options.display.max_colwidth = 300
print(df["url"].head())

0    https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.Daily+Average@40A+4080'
1        https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.discharge@40A+9000'
2              https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Stage.working@41K+4000'
3          https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Stage.working-old@41H+2000'
4    https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.Daily+Average@76E+3000'
Name: url, dtype: object


In [54]:
#Perform final field calculations on pandas DataFrame

end_df = df[["about_uri", "url", "name", "provider_code", "parameter_id", "parameter_name", "parameter_group"]]
end_df.head()
end_df.to_csv("C:\\Users\\ewiggans\\Desktop\\GeoConnexMap\\MontanaGage\\MT_download_table.csv")
#Export and write to new CSV
print("Exported to csv")

Exported to csv


### Perform final calculations and exports

In [55]:
import json
import requests
from shapely.geometry import shape, Point


def get_comid_intersect(geom):
    # Convert the input geom to GeoJSON using Shapely
    point = Point(geom)

    # Convert the Point to GeoJSON
    geom_geojson = shape(point).__geo_interface__

    url = 'https://nhdpv2-census.internetofwater.app/collections/2020/items?filter-lang=cql-json'
    filter_ = {
        'intersects': [
            {'property': 'shape'},
            geom_geojson  # Use the Shapely-converted GeoJSON
        ]
    }
    headers = {
        'Content-Type': 'application/query-cql-json'
    }
    r = requests.post(url, headers=headers, json=filter_)
    fc = r.json()
    if 'features' in fc and len(fc['features']) > 0:
        feature = fc['features'][0]
        return feature['properties']['featureid']
    else:
        return None

# Update Cursor
with arcpy.da.UpdateCursor(MT_gage, ["Shape", "comid"]) as cursor:
    for row in cursor:
        geom = row[0]
        comid = get_comid_intersect(geom)
        if comid is not None:
            print(comid, end='\r', flush=True)
            row[1] = comid
            cursor.updateRow(row)


print("comid calculated")

comid calculated


In [56]:
# Read in the CSV file as a geodatabase table
csv_table = r"C:\\Users\\ewiggans\\Desktop\\GeoConnexMap\\MontanaGage\\nhdpv2_lookup.csv"
csv_table_name = "NHDPV2_Lookup"
arcpy.TableToTable_conversion(csv_table, arcpy.env.workspace, csv_table_name)

# Add a new field 'comid_text' to the CSV table with a data type of TEXT
arcpy.AddField_management(csv_table_name, "comid_text", "TEXT")

# Calculate the 'comid_text' field by copying the values from the 'comid' field
expression = "!comid!"
codeblock = ""
arcpy.CalculateField_management(csv_table_name, "comid_text", expression, "PYTHON3", codeblock)

print("Added 'comid_text' field and set its values equal to 'comid' as text.")


Added 'comid_text' field and set its values equal to 'comid' as text.


In [57]:

# Join the CSV table to the MT gage feature class based on the "comid" field
arcpy.AddJoin_management(MT_gage, "comid", csv_table_name, "comid_text", "KEEP_COMMON")

# Calculate the "mainstem_uri" field to be equal to the "uri" field in the CSV table
expression = "!{}.uri!".format(csv_table_name)
arcpy.CalculateField_management(MT_gage, "mainstem_uri", expression, "PYTHON3")

# Remove the join to the CSV table
arcpy.RemoveJoin_management(MT_gage, csv_table_name)
print("Complete")


Complete


In [58]:
# Define the output GeoJSON file path
output_geojson_file = r"C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_gages.geojson"

# Export the feature class to GeoJSON
arcpy.FeaturesToJSON_conversion(in_features = MT_gage, 
                                out_json_file = output_geojson_file,
                                geoJSON = "GEOJSON")
print(f"Exported {MT_gage} to {output_geojson_file} in GeoJSON format.")

Exported MT_gages to C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_gages.geojson in GeoJSON format.


In [59]:
import geopandas as gpd

def group_data(gdf, gpkg_file, csv_file):
    # Make mapping dictionary
    mapping = {}
    df = pd.read_csv(csv_file)
    for index, row in df.iterrows():
        location = row['about_uri']
        data = row.drop(['about_uri'])
        if location not in mapping:
            mapping[location] = []
        mapping[location].append(data.to_dict())
    #print(mapping)
    # Apply mapping
    for index, row in gdf.iterrows():
        location = row['uri']
        if location in mapping:
            gdf.at[index, 'data'] = \
                json.dumps(mapping[location])
     #   print(location)
    gdf.to_file(gpkg_file, driver="GPKG")

gdf = gpd.read_file(r"C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_gages.geojson")
csv_file = r"C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_download_table.csv"
gpkg_file = r"C:\Users\ewiggans\Desktop\GeoConnexMap\MontanaGage\MT_gages.gpkg"
group_data(gdf, gpkg_file, csv_file)
print("Complete")

{'https://geoconnex.us/mtdnrc/gages/40A4080': [{'Unnamed: 0': 0, 'url': "https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.Daily+Average@40A+4080'", 'name': 'Discharge.Daily Average@40A 4080', 'provider_code': 'mtdnrc', 'parameter_id': 'mtdnrc-Discharge', 'parameter_name': 'Discharge.Daily Average', 'parameter_group': 'Discharge'}, {'Unnamed: 0': 467, 'url': "https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Discharge.discharge@40A+4080'", 'name': 'Discharge.discharge@40A 4080', 'provider_code': 'mtdnrc', 'parameter_id': 'mtdnrc-Discharge', 'parameter_name': 'Discharge.discharge', 'parameter_group': 'Discharge'}, {'Unnamed: 0': 468, 'url': "https://gis.dnrc.mt.gov/arcgis/rest/services/WRD/WMB_StAGE/MapServer/3/query?where=SensorCode='Stage.working@40A+4080'", 'name': 'Stage.working@40A 4080', 'provider_code': 'mtdnrc', 'parameter_id': 'mtdnrc-Stage', 'parameter_name': 'Stage.working', 'p


https://geoconnex.us/mtdnrc/gages/40A10000
https://geoconnex.us/mtdnrc/gages/40A1500
https://geoconnex.us/mtdnrc/gages/40A2000
https://geoconnex.us/mtdnrc/gages/40A3000
https://geoconnex.us/mtdnrc/gages/40A4080
https://geoconnex.us/mtdnrc/gages/40A5000
https://geoconnex.us/mtdnrc/gages/40A7000
https://geoconnex.us/mtdnrc/gages/40A8080
https://geoconnex.us/mtdnrc/gages/40A8100
https://geoconnex.us/mtdnrc/gages/40A9000
https://geoconnex.us/mtdnrc/gages/40B06000
https://geoconnex.us/mtdnrc/gages/40F00900
https://geoconnex.us/mtdnrc/gages/40F01100
https://geoconnex.us/mtdnrc/gages/40F01700
https://geoconnex.us/mtdnrc/gages/40F01800
https://geoconnex.us/mtdnrc/gages/40F01900
https://geoconnex.us/mtdnrc/gages/40I10000
https://geoconnex.us/mtdnrc/gages/40J08900
https://geoconnex.us/mtdnrc/gages/40J09000
https://geoconnex.us/mtdnrc/gages/40J09100
https://geoconnex.us/mtdnrc/gages/40L-10050013
https://geoconnex.us/mtdnrc/gages/40R05000
https://geoconnex.us/mtdnrc/gages/41A02000
https://geoconn

https://geoconnex.us/mtdnrc/gages/43BJ07800
https://geoconnex.us/mtdnrc/gages/43BJ09000
https://geoconnex.us/mtdnrc/gages/43BV02000
https://geoconnex.us/mtdnrc/gages/43BV04000
https://geoconnex.us/mtdnrc/gages/43BV05000
https://geoconnex.us/mtdnrc/gages/43BV07000
https://geoconnex.us/mtdnrc/gages/43BV08000
https://geoconnex.us/mtdnrc/gages/43BV09000
https://geoconnex.us/mtdnrc/gages/43D01900
https://geoconnex.us/mtdnrc/gages/43Q05900
https://geoconnex.us/mtdnrc/gages/43Q06300
https://geoconnex.us/mtdnrc/gages/43Q06400
https://geoconnex.us/mtdnrc/gages/46J07500
https://geoconnex.us/mtdnrc/gages/46J07800
https://geoconnex.us/mtdnrc/gages/76E1500
https://geoconnex.us/mtdnrc/gages/76E2000
https://geoconnex.us/mtdnrc/gages/76E2100
https://geoconnex.us/mtdnrc/gages/76E2200
https://geoconnex.us/mtdnrc/gages/76E3000
https://geoconnex.us/mtdnrc/gages/76E4000
https://geoconnex.us/mtdnrc/gages/76F02000
https://geoconnex.us/mtdnrc/gages/76F02900
https://geoconnex.us/mtdnrc/gages/76F03500
https://g